In [2]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
import pandas as pd
import re

In [1]:
# imports for langchain
from langchain_community.document_loaders import DirectoryLoader, UnstructuredWordDocumentLoader,UnstructuredExcelLoader, TextLoader, PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import numpy as np
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.retriever import create_retriever_tool

In [3]:
load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")

In [4]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "openai/gpt-4o-mini"#"mistralai/mistral-small-3.2-24b-instruct:free"
db_name = "ken_vector_db"

In [6]:

# --- Configuration ---
# Set the path to the folder containing your files
folder = "my_knowledge_base/"

# --- Install Required Libraries ---
# You'll need to install the loaders for each file type.
# For this example, you would run:
# pip install pypdf unstructured python-docx

# --- Create Loaders for Each File Type ---
# 1. Loader for Markdown (.md) and Text (.txt) files
print("Loading Markdown and text files...")


txt_loader = DirectoryLoader(
    folder,
    glob="**/*.txt",
    loader_cls=TextLoader
)

md_loader = DirectoryLoader(
    folder,
    glob="**/*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

# 2. Loader for PDF files
print("Loading PDF files...")
pdf_loader = DirectoryLoader(
    folder,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

# 3. Loader for Word documents (.docx)
print("Loading Word documents...")
docx_loader = DirectoryLoader(
    folder,
    glob="**/*.docx",
    loader_cls=UnstructuredWordDocumentLoader
)

#4. Loader for Word Excel documents (.xlsx)

# print("Loading Excel documents...")
# xlsx_loader = DirectoryLoader(
#     folder,
#     glob="**/*.xlsx",
#     loader_cls=UnstructuredExcelLoader
# )

print("Loading Excel documents...")
xlsx_files = glob.glob(os.path.join(folder, "**/*.xlsx"), recursive=True)
pandas_docs = []

for xlsx_file in xlsx_files:
    print(f"Processing: {os.path.basename(xlsx_file)}")
    df = pd.read_excel(xlsx_file)
    
    # Method 1: Create structured text for each row
    for index, row in df.iterrows():
        # Create a nicely formatted text representation of each row
        content = ""
        for col in df.columns:
            if pd.notna(row[col]):  # Only include non-null values
                content += f"{col}: {row[col]}\n"
        
        # Create a document for each row
        doc = Document(
            page_content=content.strip(),
            metadata={
                "source": xlsx_file,
                "row_index": index
            }
        )
        pandas_docs.append(doc)

# --- Load Documents from All Sources ---
# Initialize a list to hold all documents
all_documents = []

# Load documents from each loader and add them to the list
all_documents.extend(txt_loader.load())
all_documents.extend(md_loader.load())
all_documents.extend(pdf_loader.load())
all_documents.extend(docx_loader.load())
#all_documents.extend(xlsx_loader.load())
all_documents.extend(pandas_docs) 

print(f"Finished loading. Found a total of {len(all_documents)} documents.")

# Now, 'all_documents' contains all the content from your files,
# ready to be processed for your knowledge worker.

Loading Markdown and text files...
Loading PDF files...
Loading Word documents...
Loading Excel documents...
Processing: Hotel_Inventory_Tracking.xlsx
Finished loading. Found a total of 701 documents.


In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(all_documents)

Created a chunk of size 2924, which is longer than the specified 1000


In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 747 documents


In [9]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


In [10]:
from langchain_core.callbacks import StdOutCallbackHandler # For debugging purposes
# Your existing LLM setup
llm = ChatOpenAI(
    model=MODEL,
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    max_tokens=1000
)

# Your yam prediction function (keep as is)
def predict_yam_portion(weight_kg: float) -> dict:
    """Predict yam portions given weight in kg, based on Kenneth's regression model."""
    intercept = -0.9256
    slope = 1.9924
    se_slope = 0.314
    t_value = 2.03  # ~95%
    
    predicted = intercept + slope * weight_kg
    margin = t_value * se_slope * weight_kg
    
    ci = (predicted - margin, predicted + margin)
    pi = (predicted - 2*margin, predicted + 2*margin)
    
    return {
        "predicted": round(predicted, 2),
        "confidence_interval": (round(ci[0], 2), round(ci[1], 2)),
        "prediction_interval": (round(pi[0], 2), round(pi[1], 2))
    }

# Fixed tool function with better input parsing and formatting
def predict_yam_portion_tool(weight_input: str) -> str:
    """Predict yam portions given weight in kg, based on Kenneth's regression model."""
    try:
        # Extract just the number from inputs like "3kg", "3.5", "4 kg", etc.
        weight_match = re.search(r'(\d+(?:\.\d+)?)', weight_input)
        if not weight_match:
            return "Please provide a valid numeric weight (e.g., '3', '3.5', or '4kg')."
        
        weight = float(weight_match.group(1))
        result = predict_yam_portion(weight)
        
        return f"""Based on Kenneth's yam portioning analysis:

For a yam weighing {weight}kg:
• Predicted portion size: {result['predicted']} servings
• Confidence interval (95%): {result['confidence_interval'][0]} to {result['confidence_interval'][1]} servings  
• Prediction interval: {result['prediction_interval'][0]} to {result['prediction_interval'][1]} servings

This prediction is based on the regression model (R² = 0.755) from Kenneth's operational analysis at Zecool Hotels."""
        
    except Exception as e:
        return f"Error calculating yam portion: {str(e)}"

# Create the yam prediction tool
yam_tool = Tool(
    name="predict_yam_portion",
    description="Calculate predicted yam portion size for a given weight in kg. Use this when users ask about yam portioning, serving calculations, or yam weight analysis.",
    func=predict_yam_portion_tool
)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()#search_kwargs={"k": 25}

# Create a retriever tool for your knowledge base
retriever_tool = create_retriever_tool(
    retriever,
    "knowledge_base_search",
    "Search the knowledge base for information about hotel operations, inventory, procedures, and Kenneth's work documentation."
)

# Combine all tools
tools = [yam_tool, retriever_tool]

# Create the agent prompt with emphasis on both RAG and tools
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are Kenneth's comprehensive AI assistant. You help with both information retrieval and specialized calculations.

CORE FUNCTIONALITY:
1. PRIMARY: Search Kenneth's knowledge base for hotel operations, inventory, procedures, academic work, and personal information
2. SPECIALIZED: Use the yam portioning tool for weight-based serving calculations

APPROACH:
- For yam portioning questions → Use the prediction tool
- For all other questions → Search the knowledge base first  
- For complex queries → Use both tools as needed
- Always be conversational and reference Kenneth's work when relevant

You should ALWAYS try to provide helpful information, whether through knowledge base search or specialized tools."""),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

# Set up memory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Create the agent using the newer tool-calling approach
agent = create_tool_calling_agent(llm, tools, prompt)

# Create agent executor with memory
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# # putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
# conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)#,
#                                                           #callbacks=[StdOutCallbackHandler()])

# Updated chat function
def chat(message, history):
    try:
        result = agent_executor.invoke({"input": message})
        return result["output"]
    except Exception as e:
        return f"Sorry, I encountered an error: {str(e)}"

# Test both functionalities
if __name__ == "__main__":
    # Test yam calculation
    print("=== Testing Yam Calculation ===")
    response1 = chat("What is the portion for a yam of 3kg?", [])
    print("Yam query response:", response1)
    print("\n" + "="*50 + "\n")
    
    # Test RAG retrieval  
    print("=== Testing RAG Retrieval ===")
    response2 = chat("What is the reorder level for swan water?", [])
    print("RAG query response:", response2)

C:\Users\KENNETH MARK\AppData\Local\Temp\ipykernel_11520\3042784485.py:95: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


=== Testing Yam Calculation ===


> Entering new AgentExecutor chain...

Invoking: `predict_yam_portion` with `3`


Based on Kenneth's yam portioning analysis:

For a yam weighing 3.0kg:
• Predicted portion size: 5.05 servings
• Confidence interval (95%): 3.14 to 6.96 servings  
• Prediction interval: 1.23 to 8.88 servings

This prediction is based on the regression model (R² = 0.755) from Kenneth's operational analysis at Zecool Hotels.For a yam weighing 3.0kg, the predicted portion size is approximately **5.05 servings**. The 95% confidence interval for this estimate ranges from **3.14 to 6.96 servings**, while the prediction interval is between **1.23 and 8.88 servings**. 

This analysis is based on Kenneth's operational work at Zecool Hotels. If you need more details or have questions about the serving size, feel free to ask!

> Finished chain.
Yam query response: For a yam weighing 3.0kg, the predicted portion size is approximately **5.05 servings**. The 95% confidence interval fo

In [13]:
# Updated chat function
def chat(message, history):
    try:
        result = agent_executor.invoke({"input": message})
        return result["output"]
    except Exception as e:
        return f"Sorry, I encountered an error: {str(e)}"

In [14]:
# And in Gradio:
view = gr.ChatInterface(chat, type="messages")
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.




> Entering new AgentExecutor chain...
Hello! How can I help you today?

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `predict_yam_portion` with `3`


Based on Kenneth's yam portioning analysis:

For a yam weighing 3.0kg:
• Predicted portion size: 5.05 servings
• Confidence interval (95%): 3.14 to 6.96 servings  
• Prediction interval: 1.23 to 8.88 servings

This prediction is based on the regression model (R² = 0.755) from Kenneth's operational analysis at Zecool Hotels.For a yam weighing 3.0kg, the predicted portion size is approximately **5.05 servings**. The confidence interval for this estimate ranges from **3.14 to 6.96 servings**, while the prediction interval is between **1.23 and 8.88 servings**. 

If you have any more questions or need further assistance, feel free to ask!

> Finished chain.
